In [12]:
# Imports
import pandas as pd
import numpy as np
import time
import psycopg2
from datetime import datetime
import statsmodels.api as sm
# import pymssql

In [14]:
# Helper Functions

# Convert to integer
def int_convert(value):
    if pd.notna(value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return None
    return None

# Handle Null Values
def handle_na(value):
    if pd.notna(value):
        return value
    else:
        return None
    
# Determine winner_ou
def get_winner_ou(total_score, over_under_line):
    if total_score > over_under_line:
        return 'over'
    elif total_score < over_under_line:
        return 'under'
    else:
        return 'push'

# Determine winner_line
def get_winner_line(score_home, score_away, spread_favorite, team_favorite_id):
    score_diff = score_home - score_away
    
    # Special handling for pick games (spread is 0)
    if spread_favorite == 0:
        if score_diff > 0:
            return 'home'
        elif score_diff < 0:
            return 'away'
        else:
            return 'push'

    adjusted_spread = -spread_favorite
        
    if score_diff > adjusted_spread:
        return 'home'
    elif score_diff < adjusted_spread:
        return 'away'
    else:
        return 'push'

# Calculate commission based on bet amount
def calculate_commission(bet_amount):
    """
    - 10% on first $1,000
    - 8% on next $4,000
    - 6% on remaining amount
    """
    commission = 0
    
    # First $1,000
    if bet_amount <= 1000:
        commission = bet_amount * 0.10
    else:
        commission = 1000 * 0.10
        remaining = bet_amount - 1000
        
        # Next $4,000
        if remaining <= 4000:
            commission += remaining * 0.08
        else:
            commission += 4000 * 0.08
            remaining = remaining - 4000
            
            # Anything over $5,000
            commission += remaining * 0.06
            
    return round(commission, 2)

# Bet result
def determine_bet_result(bet_on, winner_line, winner_ou, team_id_home, team_id_away):    
    if bet_on == 'over':
        if winner_ou == 'over':
            return 'win'
        elif winner_ou == 'under':
            return 'loss'
        else:
            return 'push'
    elif bet_on == 'under':
        if winner_ou == 'under':
            return 'win'
        elif winner_ou == 'over':
            return 'loss'
        else:
            return 'push'
    elif bet_on == 'push':
        if winner_line == 'push':
            return 'win'
        else:
            return 'loss'
    else:  # Team bet
        cursor.execute("SELECT DISTINCT team_id FROM team_history WHERE name = %s;", (bet_on,))
        bet_team_id = cursor.fetchone()
        if bet_team_id:
            bet_team_id = bet_team_id[0]
            
            # Home or away team
            if bet_team_id == team_id_home:
                if winner_line == 'home':
                    return 'win'
                elif winner_line == 'away':
                    return 'loss'
                else:
                    return 'push'
            else:
                if winner_line == 'away':
                    return 'win'
                elif winner_line == 'home':
                    return 'loss'
                else:
                    return 'push'
        else:
            print(f"Warning: Could not find team ID for bet on {bet_on}")
            return None

In [16]:
# Establish connections
conn = psycopg2.connect(
    database="iwdm", 
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)
cursor = conn.cursor()

# pymssql_conn = pymssql.connect(
#     server='stairwaytoheaven.usu.edu',
#     user='5330user',
#     password='pipelinesnow',
#     database='ironwill'
# )
# pymssql_cursor = pymssql_conn.cursor()

In [18]:
# Drop all tables in the correct order, considering foreign key dependencies
try:
    cursor.execute('''
    DROP TABLE IF EXISTS bets;
    DROP TABLE IF EXISTS games;
    ''')
    conn.commit()
except Exception as e:
    print(f"Error: {e}")

    
# Games
cursor.execute('''
CREATE TABLE IF NOT EXISTS games (
    game_id VARCHAR(50) PRIMARY KEY,
    schedule_date DATE NOT NULL,
    schedule_season SMALLINT NOT NULL,
    schedule_week SMALLINT NOT NULL,
    schedule_playoff BOOLEAN NOT NULL,
    playoff_type VARCHAR(20),
    team_id_home VARCHAR(5) NOT NULL,
    team_id_away VARCHAR(5) NOT NULL,
    score_home SMALLINT,
    score_away SMALLINT,
    team_id_favorite VARCHAR(5),
    spread_favorite DECIMAL(4,1),
    over_under_line DECIMAL(4,1),
    stadium_id INTEGER,
    stadium_neutral BOOLEAN NOT NULL,
    weather_temperature SMALLINT,
    weather_wind_mph SMALLINT,
    weather_humidity SMALLINT,
    weather_detail VARCHAR(100),
    winner_ou VARCHAR(10),
    winner_line VARCHAR(10),
    FOREIGN KEY (team_id_home) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (team_id_away) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (team_id_favorite) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (stadium_id) 
        REFERENCES stadiums (stadium_id)
    );''')
conn.commit()
print("Created games table")

cursor.execute('''
CREATE TABLE IF NOT EXISTS placed_bets (
    bet_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    game_id VARCHAR(50) NOT NULL,
    bet_amount INTEGER NOT NULL,
    bet_on VARCHAR(50) NOT NULL,
    result VARCHAR(10),
    commission_amount DECIMAL(10,2),
    FOREIGN KEY (customer_id) 
        REFERENCES customers (customer_id),
    FOREIGN KEY (game_id) 
        REFERENCES games (game_id)
    );''')
conn.commit()
print("Created placed_bets table")

Created games table
Created placed_bets table


In [5]:
# Drop all tables in the correct order, considering foreign key dependencies
try:
    cursor.execute('''
    DROP TABLE IF EXISTS placed_bets;
    DROP TABLE IF EXISTS games;
    DROP TABLE IF EXISTS stadiums;
    DROP TABLE IF EXISTS weather_stations;
    DROP TABLE IF EXISTS teams;
    DROP TABLE IF EXISTS team_history;
    DROP TABLE IF EXISTS current_teams;
    DROP TABLE IF EXISTS customers;
    ''')
    conn.commit()
except Exception as e:
    print(f"Error: {e}")

In [6]:
# Create Tables
# Customers
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    age INTEGER NOT NULL,
    customer_type VARCHAR(10) NOT NULL,
    customer_since INTEGER NOT NULL,
    income INTEGER NOT NULL,
    household_size INTEGER NOT NULL,
    mode_color VARCHAR(10) NOT NULL
    );''')
conn.commit()
print("Created customers table")

# Current Teams
cursor.execute('''
CREATE TABLE IF NOT EXISTS current_teams (
    team_id VARCHAR(5) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    name_short VARCHAR(50) NOT NULL,
    team_id_pfr VARCHAR(5) NOT NULL,
    conference VARCHAR(5),
    division VARCHAR(20)
    );''')
conn.commit()
print("Created current teams table")

# Team History
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_history (
    team_history_id SERIAL PRIMARY KEY,
    team_id VARCHAR(5) NOT NULL,
    name VARCHAR(100) NOT NULL,
    name_short VARCHAR(50) NOT NULL,
    team_id_pfr VARCHAR(5) NOT NULL,
    conference VARCHAR(5),
    division VARCHAR(20),
    FOREIGN KEY (team_id) REFERENCES current_teams (team_id)
    );''')
conn.commit()
print("Created team history table")

# Weather Stations
cursor.execute('''
CREATE TABLE IF NOT EXISTS weather_stations (
    weather_station_id CHAR(11) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    state CHAR(2),
    country CHAR(2),
    latitude DECIMAL(8,5) NOT NULL,
    longitude DECIMAL(8,5) NOT NULL,
    elevation DECIMAL(5,1) NOT NULL
    );''')
conn.commit()
print("Created weather stations table")

# Stadiums
cursor.execute('''
CREATE TABLE IF NOT EXISTS stadiums (
    stadium_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    stadium_open SMALLINT,
    stadium_close INTEGER,
    stadium_type VARCHAR(20),
    street_address VARCHAR(100),
    city VARCHAR(50),
    state CHAR(2),
    postal_code VARCHAR(10),
    country CHAR(2),
    weather_station_id CHAR(11),
    stadium_weather_station_code INTEGER,
    stadium_weather_type VARCHAR(20),
    surface VARCHAR(50),
    FOREIGN KEY (weather_station_id) 
        REFERENCES weather_stations (weather_station_id)
    );''')
conn.commit()
print("Created stadiums table")

# Games
cursor.execute('''
CREATE TABLE IF NOT EXISTS games (
    game_id VARCHAR(50) PRIMARY KEY,
    schedule_date DATE NOT NULL,
    schedule_season SMALLINT NOT NULL,
    schedule_week SMALLINT NOT NULL,
    schedule_playoff BOOLEAN NOT NULL,
    playoff_type VARCHAR(20),
    team_id_home VARCHAR(5) NOT NULL,
    team_id_away VARCHAR(5) NOT NULL,
    score_home SMALLINT,
    score_away SMALLINT,
    team_id_favorite VARCHAR(5),
    spread_favorite DECIMAL(4,1),
    over_under_line DECIMAL(4,1),
    stadium_id INTEGER,
    stadium_neutral BOOLEAN NOT NULL,
    weather_temperature SMALLINT,
    weather_wind_mph SMALLINT,
    weather_humidity SMALLINT,
    weather_detail VARCHAR(100),
    winner_ou VARCHAR(10),
    winner_line VARCHAR(10),
    FOREIGN KEY (team_id_home) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (team_id_away) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (team_id_favorite) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (stadium_id) 
        REFERENCES stadiums (stadium_id)
    );''')
conn.commit()
print("Created games table")

# Bets
cursor.execute('''
CREATE TABLE IF NOT EXISTS placed_bets (
    bet_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    game_id VARCHAR(50) NOT NULL,
    bet_amount INTEGER NOT NULL,
    bet_on VARCHAR(50) NOT NULL,
    result VARCHAR(10),
    commission_amount DECIMAL(10,2),
    FOREIGN KEY (customer_id) 
        REFERENCES customers (customer_id),
    FOREIGN KEY (game_id) 
        REFERENCES games (game_id)
    );''')
conn.commit()
print("Created placed_bets table")

Created customers table
Created current teams table
Created team history table
Created weather stations table
Created stadiums table
Created games table
Created bets table


In [7]:
# Ingest Weather Stations

stadiums_df = pd.read_csv('nfl_stadiums.csv')
weather_stations_df = stadiums_df[['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION']]
df = weather_stations_df.dropna()

# Get current stations and convert to list
current_weather_stations = pd.read_sql("SELECT weather_station_id FROM weather_stations;", conn)
cws = current_weather_stations['weather_station_id'].tolist()

for x in df.index:
    weather_station_id = df['STATION'].loc[x]
    
    # Only proceed if station is new
    if weather_station_id not in cws:
        cws.append(weather_station_id) # Make sure a station isn't inserted twice
        name = df['NAME'].loc[x]
        latitude = df['LATITUDE'].loc[x]
        longitude = df['LONGITUDE'].loc[x]
        elevation = df['ELEVATION'].loc[x]

        # Get state and country codes
        name, state_country = name.split(', ')
        state, country = state_country.split(' ')

        cursor.execute('''
            INSERT INTO weather_stations (weather_station_id, name, state, country, latitude, longitude, elevation)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        ''', (weather_station_id, name, state, country, latitude, longitude, elevation))
        print(f"Inserted station {weather_station_id}: {name}")
        
conn.commit()

# Verify data
df = pd.read_sql("SELECT * FROM weather_stations", conn, index_col='weather_station_id')
print(f"Number of records: {len(cws)}\n",df)

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\2151444998.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_weather_stations = pd.read_sql("SELECT weather_station_id FROM weather_stations;", conn)


Inserted station USW00094823: PITTSBURGH ASOS
Inserted station US1MOJC0028: KANSAS CITY 5.1 SE
Inserted station USC00410337: ARLINGTON SIX FLAGS
Inserted station USW00013881: CHARLOTTE DOUGLAS AIRPORT
Inserted station US1NYER0093: BUFFALO 1.5 W
Inserted station USC00238791: WEBSTER GROVES
Inserted station USW00023234: SAN FRANCISCO INTERNATIONAL AIRPORT
Inserted station US1WAKG0038: SEATTLE 3.5 NW
Inserted station USW00003871: CINCINNATI WEATHER SERVICE OFFICE CITY
Inserted station USW00014820: CLEVELAND HOPKINS INTERNATIONAL AIRPORT
Inserted station USW00023062: DENVER STAPLETON
Inserted station USW00093837: JACKSONVILLE NAS
Inserted station USC00186350: NATIONAL ARBORETUM DC
Inserted station USC00190860: BROCKTON
Inserted station USW00014734: NEWARK LIBERTY INTERNATIONAL AIRPORT
Inserted station USW00012839: MIAMI INTERNATIONAL AIRPORT
Inserted station USW00012842: TAMPA INTERNATIONAL AIRPORT
Inserted station USW00014898: GREEN BAY AUSTIN STRAUBEL INTERNATIONAL AIRPORT
Inserted stati

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\2151444998.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM weather_stations", conn, index_col='weather_station_id')


In [8]:
# Ingest Stadiums

df = pd.read_csv('nfl_stadiums.csv')

# Get current stadiums and convert to list
current_stadiums = pd.read_sql("SELECT name FROM stadiums;", conn)
cs = current_stadiums['name'].tolist()

for x in df.index:
    name = df['stadium_name'].loc[x]
    
    # Only proceed if stadium is new
    if name not in cs:
        cs.append(name)  # Make sure a stadium isn't inserted twice
        location = df['stadium_location'].loc[x]
        stadium_open = df['stadium_open'].loc[x]
        stadium_close = df['stadium_close'].loc[x]
        stadium_type = df['stadium_type'].loc[x]
        address = df['stadium_address'].loc[x]
        stadium_weather_station_code = df['stadium_weather_station_code'].loc[x]
        weather_station_id = df['STATION'].loc[x]
        stadium_weather_type = df['stadium_weather_type'].loc[x]
        surface = df['stadium_surface'].loc[x]
        street_address = city = state = postal_code = country = None

        ### cleaning
        ## address --> street_address, city, state, zip, country
        # Get city/state/country from location
        if pd.notna(location):
            if 'UK' in location:
                city, country = location.split(', ')
            elif 'MX' in str(stadium_weather_station_code):
                city = location.split(', ')[0]
                country = 'MX'
            elif 'Canada' in location:
                city = location.split(', ')[0]
                country = 'CA'
            elif 'Germany' in location:
                city = location.split(', ')[0]
                country = 'DE'
            elif ',' in location:  # US locations
                city, state = location.split(', ')
                country = 'US'
        # Parse address if it exists
        if pd.notna(address):
            if country == 'UK':
                parts = address.split(', ')
                street_address = parts[0]
                postal_code = parts[1].replace('London', '').strip()
            elif country == 'MX': # no clue how to handle this one
                parts = address.split(', ')
                street_address = parts[0]
            elif country == 'CA':
                parts = address.split(', ')
                street_address = parts[0]
                state = parts[2].split(' ')[0]
                postal_code = parts[2].split(' ')[1] + ' ' + parts[2].split(' ')[2]
            elif country == 'US':
                parts = address.split(', ')
                street_address = parts[0]
                postal_code = parts[-1].split(' ')[-1]
        
        
        ## stadium_open
        stadium_open = int_convert(stadium_open)
        
        
        ## stadium_close
        stadium_close = int_convert(stadium_close)
        
        
        ## stadium_weather_station_code
        stadium_weather_station_code = int_convert(stadium_weather_station_code)
                
                
        ## surface
        if pd.notna(surface):  # Check if surface exists before lowercase
            if 'grass' in surface.lower():
                surface = 'Grass'
            elif 'turf' in surface.lower():
                surface = 'Turf'
        else:
            surface = None
            
        
        ## weather_station_id
        weather_station_id = handle_na(weather_station_id)
        
        
        ## stadium_type
        stadium_type = handle_na(stadium_type)
            
            
        ## stadium_weather_type
        stadium_weather_type = handle_na(stadium_weather_type)
          
            
        ### Ingest
        cursor.execute('''
            INSERT INTO stadiums (name, stadium_open, stadium_close, stadium_type, street_address, city, state, postal_code, country, weather_station_id, stadium_weather_station_code, stadium_weather_type, surface)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        ''', (name, stadium_open, stadium_close, stadium_type, street_address, city, state, postal_code, country, weather_station_id, stadium_weather_station_code, stadium_weather_type, surface))
        print(f"Inserted stadium: {name}")

conn.commit()

# Verify data
df = pd.read_sql("SELECT * FROM stadiums", conn, index_col='stadium_id')
print(f"Number of records: {len(cs)}\n", df)

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\3946041491.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_stadiums = pd.read_sql("SELECT name FROM stadiums;", conn)


Inserted stadium: Acrisure Stadium
Inserted stadium: Alamo Dome
Inserted stadium: Allegiant Stadium
Inserted stadium: Allianz Arena
Inserted stadium: Alltel Stadium
Inserted stadium: Alumni Stadium
Inserted stadium: Anaheim Stadium
Inserted stadium: Arrowhead Stadium
Inserted stadium: AT&T Stadium
Inserted stadium: Atlanta-Fulton County Stadium
Inserted stadium: Balboa Stadium
Inserted stadium: Bank of America Stadium
Inserted stadium: Bills Stadium
Inserted stadium: Busch Memorial Stadium
Inserted stadium: Caesars Superdome
Inserted stadium: Candlestick Park
Inserted stadium: CenturyLink Field
Inserted stadium: Cinergy Field
Inserted stadium: Cleveland Municipal Stadium
Inserted stadium: Cotton Bowl
Inserted stadium: Cowboys Stadium
Inserted stadium: Dolphin Stadium
Inserted stadium: Edward Jones Dome
Inserted stadium: Empower Field at Mile High
Inserted stadium: Estadio Azteca
Inserted stadium: EverBank Field
Inserted stadium: FedEx Field
Inserted stadium: Fenway Park
Inserted stadiu

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\3946041491.py:108: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM stadiums", conn, index_col='stadium_id')


In [9]:
# Ingest Teams
df = pd.read_csv('nfl_teams.csv')

# First pass: Insert pre-2002 data
for x in df.index:
    team_id = df['team_id'].loc[x]
    name = df['team_name'].loc[x]
    name_short = df['team_name_short'].loc[x]
    team_id_pfr = df['team_id_pfr'].loc[x]
    conference = df['team_conference_pre2002'].loc[x]
    division = df['team_division_pre2002'].loc[x]

    # Clean data
    team_id = handle_na(team_id)
    name = handle_na(name)
    name_short = handle_na(name_short)
    team_id_pfr = handle_na(team_id_pfr)
    conference = handle_na(conference)
    division = handle_na(division)
    
    # Check if team_id exists
    cursor.execute('SELECT team_id FROM current_teams WHERE team_id = %s', (team_id,))
    exists = cursor.fetchone()
    
    if not exists:
        # Insert into current_teams only if it doesn't exist
        cursor.execute('''
            INSERT INTO current_teams (team_id, name, name_short, team_id_pfr, conference, division)
            VALUES (%s, %s, %s, %s, %s, %s);
        ''', (team_id, name, name_short, team_id_pfr, conference, division))
        
        # Insert into team_history
        cursor.execute('''
            INSERT INTO team_history 
            (team_id, name, name_short, team_id_pfr, conference, division)
            VALUES (%s, %s, %s, %s, %s, %s);
        ''', (team_id, name, name_short, team_id_pfr, conference, division))
        
        print(f"Inserted pre-2002 record for team: {name}")
    
# Second pass: Check for changes and update if needed
for x in df.index:
    team_id = df['team_id'].loc[x]
    name = df['team_name'].loc[x]
    name_short = df['team_name_short'].loc[x]
    team_id_pfr = df['team_id_pfr'].loc[x]
    conference = df['team_conference'].loc[x]
    division = df['team_division'].loc[x]

    # Clean data
    team_id = handle_na(team_id)
    name = handle_na(name)
    name_short = handle_na(name_short)
    team_id_pfr = handle_na(team_id_pfr)
    conference = handle_na(conference)
    division = handle_na(division)

    # Get current values
    cursor.execute('''
        SELECT name, name_short, team_id_pfr, conference, division 
        FROM current_teams 
        WHERE team_id = %s;
    ''', (team_id,))  # Fixed tuple syntax
    
    current = cursor.fetchone()
    if current:
        current_values = list(current)
        new_values = [name, name_short, team_id_pfr, conference, division]
        
        # Check if any values have changed
        if current_values != new_values:            
            cursor.execute('''
                INSERT INTO team_history 
                (team_id, name, name_short, team_id_pfr, conference, division)
                VALUES (%s, %s, %s, %s, %s, %s);
            ''', (team_id, name, name_short, team_id_pfr, conference, division))
            
            # Update current_teams
            cursor.execute('''
                UPDATE current_teams 
                SET name = %s, name_short = %s, team_id_pfr = %s, conference = %s, division = %s
                WHERE team_id = %s;
            ''', (name, name_short, team_id_pfr, conference, division, team_id))
            
            print(f"Updated record for team: {name}")

conn.commit()

# Verify data
print("\nCurrent Teams table:")
teams_df = pd.read_sql("SELECT * FROM current_teams", conn)
print(f"Number of current teams: {len(teams_df)}\n", teams_df)

print("\nTeam History table:")
history_df = pd.read_sql("SELECT * FROM team_history", conn)
print(f"Number of history records: {len(history_df)}\n", history_df)

Inserted pre-2002 record for team: Arizona Cardinals
Inserted pre-2002 record for team: Atlanta Falcons
Inserted pre-2002 record for team: Baltimore Colts
Inserted pre-2002 record for team: Baltimore Ravens
Inserted pre-2002 record for team: Boston Patriots
Inserted pre-2002 record for team: Buffalo Bills
Inserted pre-2002 record for team: Carolina Panthers
Inserted pre-2002 record for team: Chicago Bears
Inserted pre-2002 record for team: Cincinnati Bengals
Inserted pre-2002 record for team: Cleveland Browns
Inserted pre-2002 record for team: Dallas Cowboys
Inserted pre-2002 record for team: Denver Broncos
Inserted pre-2002 record for team: Detroit Lions
Inserted pre-2002 record for team: Green Bay Packers
Inserted pre-2002 record for team: Houston Oilers
Inserted pre-2002 record for team: Houston Texans
Inserted pre-2002 record for team: Jacksonville Jaguars
Inserted pre-2002 record for team: Kansas City Chiefs
Inserted pre-2002 record for team: Las Vegas Raiders
Inserted pre-2002 re

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\257198371.py:91: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  teams_df = pd.read_sql("SELECT * FROM current_teams", conn)
C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\257198371.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  history_df = pd.read_sql("SELECT * FROM team_history", conn)


In [11]:
# Ingest Customers

# df = pd.read_sql("SELECT * FROM customer_table;", pymssql_conn)
df = pd.read_csv('customer_data.csv')

# Get current customers and convert to list
current_customers = pd.read_sql('SELECT customer_id FROM customers;', conn)
cc = current_customers['customer_id'].tolist()

# Initialize timer
start_time = time.time()
last_check = start_time

# Process each customer
counter = 0
for x in df.index:
    customer_id = int_convert(df['customer_id'].loc[x])
    
    # Only proceed if customer is new
    if customer_id not in cc:
        cc.append(customer_id)
        
        customer_name = df['customer_name'].loc[x]
        age = int_convert(df['customer_age'].loc[x])
        customer_type = handle_na(df['customer_type'].loc[x])
        customer_since = int_convert(df['customer_since'].loc[x])
        income = int_convert(df['customer_income'].loc[x])
        household_size = int_convert(df['household_size'].loc[x])
        mode_color = handle_na(df['mode_color'].loc[x])
        
        # Name
        name_parts = customer_name.split(' ')
        if len(name_parts) == 2:
            first_name = name_parts[0]
            last_name = name_parts[1]
        else:
            first_name = name_parts[0]
            last_name = ' '.join(name_parts[1:])
        
        # Insert into customers table
        cursor.execute('''
            INSERT INTO customers 
            (customer_id, first_name, last_name, age, customer_type, 
             customer_since, income, household_size, mode_color)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        ''', (customer_id, first_name, last_name, age, customer_type, 
              customer_since, income, household_size, mode_color))

        counter += 1
        if counter % 100 == 0:  # Print progress every 100 records
            current_time = time.time()
            elapsed_since_last = current_time - last_check
            total_elapsed = current_time - start_time
    
            last_check_min = int(elapsed_since_last // 60)
            last_check_sec = int(elapsed_since_last % 60)
            total_min = int(total_elapsed // 60)
            total_sec = int(total_elapsed % 60)
    
            percent_complete = (counter / len(df)) * 100
            
            print(f"Processed {counter} customers ({percent_complete:.1f}%) | Last 100 customers: {last_check_min}:{last_check_sec:02d} | Total time: {total_min}:{total_sec:02d}")
            last_check = current_time

conn.commit()

# Final timing
end_time = time.time()
total_time = end_time - start_time
total_min = int(total_time // 60)
total_sec = int(total_time % 60)
print(f"\nTotal processing time: {total_min}:{total_sec:02d}")

# Verify data
print("\nCustomers table:")
customers_df = pd.read_sql("SELECT * FROM customers", conn)
print(f"Number of customers: {len(cc)}")
print(customers_df)

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\1053402355.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_customers = pd.read_sql('SELECT customer_id FROM customers;', conn)


Processed 100 customers (5.0%) | Last 100 customers: 0:04 | Total time: 0:04
Processed 200 customers (10.0%) | Last 100 customers: 0:04 | Total time: 0:09
Processed 300 customers (15.0%) | Last 100 customers: 0:04 | Total time: 0:14
Processed 400 customers (20.0%) | Last 100 customers: 0:04 | Total time: 0:19
Processed 500 customers (25.0%) | Last 100 customers: 0:05 | Total time: 0:24
Processed 600 customers (30.0%) | Last 100 customers: 0:05 | Total time: 0:30
Processed 700 customers (35.0%) | Last 100 customers: 0:04 | Total time: 0:35
Processed 800 customers (40.0%) | Last 100 customers: 0:04 | Total time: 0:39
Processed 900 customers (45.0%) | Last 100 customers: 0:04 | Total time: 0:44
Processed 1000 customers (50.0%) | Last 100 customers: 0:04 | Total time: 0:48
Processed 1100 customers (55.0%) | Last 100 customers: 0:04 | Total time: 0:53
Processed 1200 customers (60.0%) | Last 100 customers: 0:05 | Total time: 0:58
Processed 1300 customers (65.0%) | Last 100 customers: 0:05 | 

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\1053402355.py:76: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers_df = pd.read_sql("SELECT * FROM customers", conn)


Number of customers: 2000
      customer_id first_name last_name  age customer_type  customer_since  \
0               1   Coraline    Flores   23        online            2022   
1               2    Presley     Ortiz   41        online            2021   
2               3       Reid    Knight   40        online            2023   
3               4   Clarissa  Chandler   43         local            2023   
4               5      Isaac    Vaughn   23         phone            2023   
...           ...        ...       ...  ...           ...             ...   
1995         1996     Maxine      Pope   36         local            2020   
1996         1997    Camilla     Wolfe   46         local            2020   
1997         1998     Karina    Knight   36         local            2020   
1998         1999  Sebastian   Winters   55         local            2020   
1999         2000   Hadassah      Wade   29         local            2021   

      income  household_size mode_color  
0     1

In [20]:
# Ingest Games

# Read games data
df = pd.read_csv('spread_scores-2.csv')

# Filter for games from 2015 onwards
df = df[df['schedule_season'] >= 2015]

# Get current games and convert to list
current_games = pd.read_sql("SELECT game_id FROM games;", conn)
cg = current_games['game_id'].tolist()

TEAM_ID_CORRECTIONS = {
    'LV': 'LVR',
    }

# Initialize timer
start_time = time.time()
last_check = start_time

# Process each game
counter = 0
for x in df.index:
    # Construct game_id 
    season = str(df['schedule_season'].loc[x])
    
    weeknum = df['schedule_week'].loc[x]
    if weeknum == "Wildcard":
        playoff_type = "Wildcard"
        weeknum = "19"
    elif weeknum == "Division":
        playoff_type = "Division"
        weeknum = "20"
    elif weeknum == "Conference":
        playoff_type = "Conference"
        weeknum = "21"
    elif weeknum == "Superbowl":
        playoff_type = "Superbowl"
        weeknum = "22"
    else:
        playoff_type = None
        weeknum = str(weeknum)
        
    if len(weeknum) < 2: 
        weeknum = "0" + weeknum

    weekstring = weeknum
    weeknum = int_convert(weeknum)
    
    cursor.execute(f"SELECT DISTINCT team_id FROM team_history WHERE name = '{df['team_home'].loc[x]}';")
    home_id = cursor.fetchone()[0]
    home_id = TEAM_ID_CORRECTIONS.get(home_id, home_id)
    
    cursor.execute(f"SELECT DISTINCT team_id FROM team_history WHERE name = '{df['team_away'].loc[x]}';")
    away_id = cursor.fetchone()[0]
    away_id = TEAM_ID_CORRECTIONS.get(away_id, away_id)
    
    game_id = f"{season}{weekstring}-{home_id}-{away_id}"
    
    # Only proceed if game is new
    if game_id not in cg:
        cg.append(game_id)
        
        # Get required data
        schedule_date = datetime.strptime(df['schedule_date'].loc[x], '%m/%d/%Y')
        schedule_season = int_convert(df['schedule_season'].loc[x])
        schedule_week = weeknum
        schedule_playoff = bool(df['schedule_playoff'].loc[x])
        score_home = int_convert(df['score_home'].loc[x])
        score_away = int_convert(df['score_away'].loc[x])
        team_favorite_id = df['team_favorite_id'].loc[x]
        spread_favorite = df['spread_favorite'].loc[x]
        over_under_line = df['over_under_line'].loc[x]
        stadium_name = df['stadium'].loc[x]
        stadium_neutral = bool(df['stadium_neutral'].loc[x])
        weather_temperature = int_convert(df['weather_temperature'].loc[x])
        weather_wind_mph = int_convert(df['weather_wind_mph'].loc[x])
        weather_humidity = int_convert(df['weather_humidity'].loc[x])
        weather_detail = handle_na(df['weather_detail'].loc[x])        


        # Try to find existing stadium
        stadium_id = None
        cursor.execute("SELECT stadium_id FROM stadiums WHERE name = %s;", (stadium_name,))
        stadium_result = cursor.fetchone()

        if stadium_result:
            stadium_id = stadium_result[0]
        else:
            # Insert new stadium with minimal info and get its ID
            cursor.execute('''INSERT INTO stadiums (name) VALUES (%s);''', (stadium_name,))
            conn.commit()
            cursor.execute("SELECT stadium_id FROM stadiums WHERE name = %s;", (stadium_name,))
            stadium_id = cursor.fetchone()[0]
            print(f"Added new stadium: {stadium_name} with ID: {stadium_id}")
        
        # Handle PICK
        if team_favorite_id == 'PICK':
            team_favorite_id = home_id  # Use home team and set spread to 0 float
            spread_favorite = 0.0
        else:
            team_favorite_id = TEAM_ID_CORRECTIONS.get(team_favorite_id, team_favorite_id)
        
        # Calculate winner_ou and winner_line
        total_score = score_home + score_away
        winner_ou = get_winner_ou(total_score, over_under_line)
        winner_line = get_winner_line(score_home, score_away, spread_favorite, team_favorite_id)

        # Insert into games table
        cursor.execute('''
            INSERT INTO games (
                game_id, schedule_date, schedule_season, schedule_week, 
                schedule_playoff, playoff_type, team_id_home, team_id_away, 
                score_home, score_away, team_id_favorite, spread_favorite,
                over_under_line, stadium_id, stadium_neutral, 
                weather_temperature, weather_wind_mph, weather_humidity,
                weather_detail, winner_ou, winner_line)
            VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s);
        ''', (game_id, schedule_date, schedule_season, schedule_week,
              schedule_playoff, playoff_type, home_id, away_id,
              score_home, score_away, team_favorite_id, spread_favorite,
              over_under_line, stadium_id, stadium_neutral,
              weather_temperature, weather_wind_mph, weather_humidity,
              weather_detail, winner_ou, winner_line))

        counter += 1
        if counter % 100 == 0:  # Print progress every 100 records
            current_time = time.time()
            elapsed_since_last = current_time - last_check
            total_elapsed = current_time - start_time
    
            last_check_min = int(elapsed_since_last // 60)
            last_check_sec = int(elapsed_since_last % 60)
            total_min = int(total_elapsed // 60)
            total_sec = int(total_elapsed % 60)
    
            percent_complete = (counter / len(df)) * 100
            
            print(f"Processed {counter} games ({percent_complete:.1f}%) | Last 100 games: {last_check_min}:{last_check_sec:02d} | Total time: {total_min}:{total_sec:02d}")
            last_check = current_time

conn.commit()

# Final timing
end_time = time.time()
total_time = end_time - start_time
total_min = int(total_time // 60)
total_sec = int(total_time % 60)
print(f"\nTotal processing time: {total_min}:{total_sec:02d}")

# Verify data
print("\nGames table:")
games_df = pd.read_sql("SELECT * FROM games;", conn)
print(f"Number of games: {len(cg)}")
print(games_df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_52102/3480200415.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_games = pd.read_sql("SELECT game_id FROM games;", conn)


Processed 100 games (4.1%) | Last 100 games: 0:27 | Total time: 0:27
Processed 200 games (8.1%) | Last 100 games: 0:26 | Total time: 0:53
Processed 300 games (12.2%) | Last 100 games: 0:26 | Total time: 1:20
Processed 400 games (16.3%) | Last 100 games: 0:30 | Total time: 1:50
Processed 500 games (20.3%) | Last 100 games: 0:27 | Total time: 2:17
Processed 600 games (24.4%) | Last 100 games: 0:26 | Total time: 2:43
Processed 700 games (28.5%) | Last 100 games: 0:26 | Total time: 3:10
Processed 800 games (32.5%) | Last 100 games: 0:28 | Total time: 3:38
Processed 900 games (36.6%) | Last 100 games: 0:26 | Total time: 4:04
Processed 1000 games (40.7%) | Last 100 games: 0:26 | Total time: 4:31
Processed 1100 games (44.8%) | Last 100 games: 0:27 | Total time: 4:58
Processed 1200 games (48.8%) | Last 100 games: 0:26 | Total time: 5:25
Processed 1300 games (52.9%) | Last 100 games: 0:26 | Total time: 5:51
Processed 1400 games (57.0%) | Last 100 games: 0:26 | Total time: 6:17
Processed 1500 ga

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_52102/3480200415.py:155: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  games_df = pd.read_sql("SELECT * FROM games;", conn)


Number of games: 2458
             game_id schedule_date  schedule_season  schedule_week  \
0      201501-NE-PIT    2015-09-10             2015              1   
1      201501-ARI-NO    2015-09-13             2015              1   
2     201501-BUF-IND    2015-09-13             2015              1   
3      201501-CHI-GB    2015-09-13             2015              1   
4     201501-DAL-NYG    2015-09-13             2015              1   
...              ...           ...              ...            ...   
2453   202320-BUF-KC    2024-01-21             2023             20   
2454   202320-DET-TB    2024-01-21             2023             20   
2455   202321-BAL-KC    2024-01-28             2023             21   
2456   202321-SF-DET    2024-01-28             2023             21   
2457    202322-KC-SF    2024-02-11             2023             22   

      schedule_playoff playoff_type team_id_home team_id_away  score_home  \
0                False         None           NE          PI

In [21]:
# Ingest Bets

# Read betting data
# df = pd.read_sql("SELECT * FROM betlog;", pymssql_conn)
df = pd.read_csv('betting_data.csv')

# Get current bets and convert to list
current_bets = pd.read_sql("SELECT bet_id FROM placed_bets;", conn)
cb = current_bets['bet_id'].tolist()
GAME_ID_CORRECTIONS = {
    'LV': 'LVR',
    'JAC': 'JAX'
    }

# Initialize timer
start_time = time.time()
last_check = start_time

# Process each bet
counter = 0
for x in df.index:
    if counter >= 3000:
        break
        
    bet_id = int_convert(df['bet_id'].loc[x])
    
    # Only proceed if bet is new
    if bet_id not in cb:
        cb.append(bet_id)
        
        customer_id = int_convert(df['customer_id'].loc[x])

        # Fix game ids
        game_id = df['game_id'].loc[x]
        game_id_parts = game_id.split('-')
        if len(game_id_parts) == 3:
            season_week, team1, team2 = game_id_parts
            team1 = GAME_ID_CORRECTIONS.get(team1, team1)
            team2 = GAME_ID_CORRECTIONS.get(team2, team2)
            game_id = f"{season_week}-{team1}-{team2}"

        
        bet_amount = int_convert(df['bet_amount'].loc[x])
        bet_on = df['bet_on'].loc[x]
        
        # Get game details to determine bet result
        cursor.execute("SELECT winner_line, winner_ou, team_id_home, team_id_away FROM games WHERE game_id = %s;", (game_id,))
        game_details = cursor.fetchone()
        
        if game_details:
            winner_line, winner_ou, team_id_home, team_id_away = game_details
            
            # Get result and commission
            result = determine_bet_result(bet_on, winner_line, winner_ou, team_id_home, team_id_away)                
            commission_amount = calculate_commission(bet_amount)
            
            # Insert into bets table
            cursor.execute('''
                INSERT INTO placed_bets (bet_id, customer_id, game_id, bet_amount, bet_on, result, commission_amount)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
            ''', (bet_id, customer_id, game_id, bet_amount, bet_on, result, commission_amount))
        else:
            print(f"Warning: Could not find game details for game_id {game_id}")
    counter += 1
    if counter % 1000 == 0:  # Print progress every 1000 records
        current_time = time.time()
        elapsed_since_last = current_time - last_check
        total_elapsed = current_time - start_time

        last_check_min = int(elapsed_since_last // 60)
        last_check_sec = int(elapsed_since_last % 60)
        total_min = int(total_elapsed // 60)
        total_sec = int(total_elapsed % 60)

        percent_complete = (counter / len(df)) * 100
        
        print(f"Processed {counter} bets ({percent_complete:.1f}%) | Last 1000 bets: {last_check_min}:{last_check_sec:02d} | Total time: {total_min}:{total_sec:02d}")
        last_check = current_time

conn.commit()

# Final timing
end_time = time.time()
total_time = end_time - start_time
total_min = int(total_time // 60)
total_sec = int(total_time % 60)
print(f"\nTotal processing time: {total_min}:{total_sec:02d}")

# Verify data
print("\nBets table:")
bets_df = pd.read_sql("SELECT * FROM placed_bets;", conn)
print(f"Number of bets: {len(cb)}")
print(bets_df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_52102/4275608737.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_bets = pd.read_sql("SELECT bet_id FROM placed_bets;", conn)


Processed 1000 bets (0.8%) | Last 1000 bets: 3:04 | Total time: 3:04
Processed 2000 bets (1.6%) | Last 1000 bets: 3:05 | Total time: 6:09
Processed 3000 bets (2.4%) | Last 1000 bets: 3:03 | Total time: 9:13

Total processing time: 9:13

Bets table:


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_52102/4275608737.py:91: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bets_df = pd.read_sql("SELECT * FROM placed_bets;", conn)


Number of bets: 3000
      bet_id  customer_id         game_id  bet_amount                bet_on  \
0          1            1   202301-KC-DET        8500         Detroit Lions   
1          2            7   202301-KC-DET         350                 under   
2          3            8   202301-KC-DET       10000    Kansas City Chiefs   
3          4           23   202301-KC-DET         200                 under   
4          5           24   202301-KC-DET         400                  over   
...      ...          ...             ...         ...                   ...   
2995    2996         1963  202301-IND-JAX         100  Jacksonville Jaguars   
2996    2997         1968  202301-IND-JAX        3000                 under   
2997    2998         1979  202301-IND-JAX        6500  Jacksonville Jaguars   
2998    2999         1980  202301-IND-JAX         100  Jacksonville Jaguars   
2999    3000         1981  202301-IND-JAX         100    Indianapolis Colts   

     result  commission_amount

In [14]:
# # Create csvs of finals
# tables = ['customers', 'bets', 'games', 'team_history', 'current_teams', 'stadiums', 'weather_stations']
# for table in tables:
#     query = f"SELECT * FROM {table};"
#     df = pd.read_sql(query, conn)

#     filename = f'iwdm_{table}.csv'
#     df.to_csv(filename, index=False)
#     print(f"Successfully exported {table} to {filename}")
#     print(f"Number of records: {len(df)}")

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10660\661526783.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Successfully exported customers to iwdm_customers.csv
Number of records: 2000
Successfully exported bets to iwdm_bets.csv
Number of records: 126047
Successfully exported games to iwdm_games.csv
Number of records: 2458
Successfully exported team_history to iwdm_team_history.csv
Number of records: 62
Successfully exported current_teams to iwdm_current_teams.csv
Number of records: 32
Successfully exported stadiums to iwdm_stadiums.csv
Number of records: 120
Successfully exported weather_stations to iwdm_weather_stations.csv
Number of records: 29


In [ ]:
# Test Queries

# 1
tables = ['customers', 'placed_bets', 'games', 'team_history', 'current_teams', 'stadiums', 'weather_stations']
for table in tables:
    query = f"SELECT * FROM {table} LIMIT 5;"
    df = pd.read_sql(query, conn)
    print(f"\nFirst 5 rows from {table}:\n", df)

In [ ]:
# 2a
query = """
WITH commission_totals AS (
    SELECT 
        c.customer_id, 
        SUM(commission_amount) as total_commission
    FROM customers c
    JOIN placed_bets b ON c.customer_id = b.customer_id
    GROUP BY c.customer_id
    HAVING SUM(commission_amount) > 20000
)
SELECT 
    COUNT(*) as customers_over_20k,
    (SELECT COUNT(*) FROM customers) as total_customers,
    ROUND(CAST(COUNT(*) AS DECIMAL) / (SELECT COUNT(*) FROM customers) * 100, 2) as percentage
FROM commission_totals;
"""
df = pd.read_sql(query, conn)
print("Summary of high-commission customers:\n", df)

# 2b
query = """
SELECT 
    c.first_name,
    c.last_name,
    ROUND(CAST(SUM(commission_amount) AS DECIMAL), 2) as total_commission
FROM customers c
JOIN placed_bets b ON c.customer_id = b.customer_id
GROUP BY c.customer_id, first_name, last_name
HAVING SUM(commission_amount) > 20000
ORDER BY total_commission DESC
LIMIT 20;
"""
df = pd.read_sql(query, conn)
print("\nTop 20 commission payers:\n", df)

In [ ]:
# 3
query = """
WITH bet_stats AS (
    SELECT 
        c.customer_id,
        first_name,
        last_name,
        COUNT(*) as total_bets,
        COUNT(CASE WHEN b.result = 'win' THEN 1 END) as wins,
        SUM(CASE 
            WHEN b.result = 'win' THEN b.bet_amount
            WHEN b.result = 'loss' THEN -b.bet_amount
            ELSE 0 END) as total_winnings
    FROM customers c
    JOIN placed_bets b ON c.customer_id = b.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
    HAVING COUNT(*) >= 6
)
SELECT 
    first_name,
    last_name,
    total_bets,
    wins,
    ROUND((CAST(wins AS DECIMAL) / total_bets * 100), 2) as win_percentage,
    total_winnings
FROM bet_stats
ORDER BY win_percentage DESC, total_winnings DESC
LIMIT 10;
"""
df = pd.read_sql(query, conn)
print("Top 10 luckiest bettors:\n", df)

In [ ]:
# 4
query = """
SELECT 
    first_name,
    last_name,
    COUNT(*) as total_bets,
    COUNT(CASE WHEN result = 'win' THEN 1 END) as wins,
    SUM(CASE 
        WHEN result = 'win' THEN -(bet_amount + bet_amount)
        WHEN result = 'loss' THEN bet_amount + commission_amount
        WHEN result = 'push' THEN commission_amount END) as net_lost_for_company
FROM customers c
JOIN placed_bets b ON c.customer_id = b.customer_id
GROUP BY c.customer_id, first_name, last_name
ORDER BY net_lost_for_company
LIMIT 20;
"""
df = pd.read_sql(query, conn)
print("20 costliest customers for the sports book:\n", df)

In [ ]:
# 5
query = """
WITH weekly_games AS (
    SELECT 
        schedule_week,
        COUNT(*) as total_games,
        COUNT(CASE WHEN 
            (winner_line = 'home' AND spread_favorite > 0) OR 
            (winner_line = 'away' AND spread_favorite < 0) THEN 1 END) as winners,
        COUNT(CASE WHEN 
            (winner_line = 'away' AND spread_favorite > 0) OR
            (winner_line = 'home' AND spread_favorite < 0) THEN 1 END) as losers
    FROM games
    WHERE schedule_season = 2023 AND schedule_playoff = false
    GROUP BY schedule_week
)
SELECT 
    schedule_week,
    total_games,
    winners,
    losers,
    ROUND((CAST(winners AS DECIMAL) / total_games * 100), 2) as winner_percentage,
    ROUND((CAST(losers AS DECIMAL) / total_games * 100), 2) as loser_percentage
FROM weekly_games
ORDER BY schedule_week;
"""
df = pd.read_sql(query, conn)
print("Weekly house performance for 2023 season:\n", df)

In [ ]:
# 6
query = """
WITH team_games AS (
    SELECT 
        name as team_name,
        game_id,
        CASE 
            WHEN team_id_home = team_id THEN 'home'
            ELSE 'away' END as home_or_away,
        score_home,
        score_away,
        winner_line,
        spread_favorite,
        team_id_favorite
    FROM current_teams t
    JOIN games g ON t.team_id = g.team_id_home OR t.team_id = g.team_id_away
    WHERE schedule_season = 2023
),
team_records AS (
    SELECT 
        team_name,
        COUNT(*) AS games_played,
        COUNT(CASE WHEN 
            (home_or_away = 'home' AND score_home > score_away) OR
            (home_or_away = 'away' AND score_away > score_home) THEN 1 END) as wins,
        COUNT(CASE WHEN 
            (home_or_away = 'home' AND score_home < score_away) OR
            (home_or_away = 'away' AND score_away < score_home) THEN 1 END) as losses,
        COUNT(CASE WHEN home_or_away = winner_line THEN 1 END) as beat_spread
    FROM team_games
    GROUP BY team_name
),
betting_counts AS (
    SELECT 
        name AS team_name,
        COUNT(CASE WHEN bet_on = ct.name THEN 1 END) AS bets_for,
        COUNT(CASE WHEN bet_on != ct.name AND
            bet_on NOT IN ('over', 'under') AND
            team_id_home = team_id OR
            team_id_away = team_id THEN 1 END) as bets_against
    FROM current_teams ct
    LEFT JOIN games g ON ct.team_id = g.team_id_home OR ct.team_id = g.team_id_away
    LEFT JOIN placed_bets b ON g.game_id = b.game_id
    WHERE schedule_season = 2023
    GROUP BY team_id, name
)
SELECT 
    r.team_name,
    wins,
    losses,
    beat_spread,
    COALESCE(bets_for, 0) as bets_for,
    COALESCE(bets_against, 0) as bets_against
FROM team_records r
LEFT JOIN betting_counts b ON r.team_name = b.team_name
ORDER BY r.team_name;
"""
df = pd.read_sql(query, conn)
print("Per-team betting analysis for 2023 season:\n", df)

In [ ]:
# Test Analysis

# Query to calculate customer value and create features
query = """
WITH bet_values AS (
    SELECT 
        customer_id,
        SUM(bet_amount) AS total_bet,
        SUM(commission_amount) as total_commission,
        COUNT(*) as num_bets,
        COUNT(CASE WHEN result = 'win' THEN 1 END) as num_wins,
        SUM(CASE 
            WHEN result = 'win' THEN -(bet_amount * 2)
            WHEN result = 'loss' THEN bet_amount
            ELSE 0 END) AS bet_outcomes
    FROM placed_bets
    GROUP BY customer_id
)
SELECT 
    c.customer_id,
    age,
    income,
    household_size, 
    customer_type,
    customer_since,
    mode_color,
    first_name,
    last_name,
    num_bets,
    num_wins,
    CASE WHEN customer_type = 'local' THEN 1 ELSE 0 END as type_local,
    CASE WHEN customer_type = 'online' THEN 1 ELSE 0 END as type_online,
    CASE WHEN customer_type = 'phone' THEN 1 ELSE 0 END as type_phone,
    total_commission + bet_outcomes as customer_value
FROM customers c
LEFT JOIN bet_values bv ON c.customer_id = bv.customer_id;
"""
df = pd.read_sql(query, conn)

# Feature engineering
df['name_length'] = df['first_name'].str.len() + df['last_name'].str.len()
df['years_customer'] = 2024 - df['customer_since']
df['win_rate'] = df['num_wins'] / df['num_bets']

# A: Initial correlation matrix with all variables
initial_predictors = ['age', 'income', 'household_size', 'type_local', 'type_online', 'type_phone', 'years_customer', 'name_length', 'win_rate']
corr_matrix = df[initial_predictors].corr()
print("\nInitial Correlation Matrix:\n", corr_matrix)

# B: First regression
first_predictors = ['age', 'income', 'household_size', 'type_local']
X = df[first_predictors]
y = df['customer_value']
X = sm.add_constant(X)
model1 = sm.OLS(y, X).fit()
print("\nFirst Model Results:")
print(model1.summary())

# C: Second regression
second_predictors = ['win_rate', 'years_customer', 'type_local', 'household_size']
X2 = df[second_predictors]
y = df['customer_value']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y, X2).fit()
print("\nSecond Model Results:")
print(model2.summary())

print(f"""
Regression Analysis Summary:
--------------------------
1. Correlation Analysis:
    - High collinearity detected between customer types (type_local, type_online, type_phone)
    - No other concerning correlations found among independent variables
    - Removed type_online and type_phone due to collinearity with type_local

2. First Model:
    - Variables: age, income, household_size, type_local
    - Adjusted R-squared: {model1.rsquared_adj:.4f}
    - Key findings:
        * Age not significant (p={model1.pvalues['age']:.4f})
        * All other variables significant at p<0.05
        * Negative relationship with income and household_size
        * Positive relationship with type_local

3. Second Model:
    - Variables: win_rate, years_customer, type_local, household_size
    - Adjusted R-squared: {model2.rsquared_adj:.4f}
    - Key findings:
        * Win rate and household_size significant at p<0.05
        * Years_customer (p=0.229) and type_local (p=0.093) not significant
        * Negative relationships with all variables except type_local
        * Household size remains consistently significant across models

4. Model Comparison:
    - First model Adj R-squared: {model1.rsquared_adj:.4f}
    - Second model Adj R-squared: {model2.rsquared_adj:.4f}
    - First model shows better explanatory power
    - Both models identify significant predictors of customer value
""")

In [30]:
# Close cursor and connection
cursor.close()
conn.close()

# pymssql_cursor.close() 
# pymssql_conn.close()

# Might need to fix teams/team history connection to games

In [ ]:
# # Establish connections
# conn = psycopg2.connect(
#     database="iwdm", 
#     user='dw_chancewiese',
#     password='Spikeball2020',
#     host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
#     port='5432'
# )
# cursor = conn.cursor()

# # pymssql_conn = pymssql.connect(
# #     server='stairwaytoheaven.usu.edu',
# #     user='5330user',
# #     password='pipelinesnow',
# #     database='ironwill'
# # )
# # pymssql_cursor = pymssql_conn.cursor()